## *Nagłówek sprawozdania*

*kto, kiedy, na jakich zajęciach, itp., itd. - do uzypełnienia!*

# Analiza obrazu - detekcja twarzy

Do realizacji zadań wykorzystywane są biblioteki OpenCV, dlib oraz InsightFace (wymaga dodatkowo pakietu onnxruntime), ponadto wykorzystywana jest biblioteka PIL/pillow do wyświetlania obrazów w notatniku. Biblioteki te muszą być zainstalowane w środowisku Python, odpowiedni plik <tt>requirements.txt</tt> został udostępniony razem z notatnikiem.

Do wykonania notatnika potrzebne są:
- obraz testowy: <tt>2_Demonstration_Demonstration_Or_Protest_2_58.jpg</tt>
- model detektora Haar: <tt>haarcascade_frontalface_default.xml</tt>
- model detektora MMOD (sieć neuronowa z biblioteki dlib): <tt>mmod_human_face_detector.dat</tt>
- modele detektora twarzy (siec neuronowa) z biblioteki insightface: katalog <tt>insightface</tt>

Wszystkie potrzebne pliki powinny znajdować się w tym samym katalogu co notatnik.

> Pracując w środowisku Google Colab należy potrzebne pliki wejściowe wgrać do środowiska notatnika - można to zrobić za pomocą interfejsu środowiska w przeglądarce, lub za pomocą widgetu 'upload'. Alternatywnie można wczytywać dane z własnego dysku Google Drive, co może być czasami przydatne - wgrywanie plików do środowiska Colab może być czasochłonne (w przypadku dużej liczby plików lub dużego ich rozmiaru), ponadto jest wymagane każdorazowo po inicjalizacji środowiska (przykłady poniżej).

In [ ]:
# from google.colab import files
# uploaded = files.upload()

# # alternatywnie - czytanie plików z dysku Google Drive, w tym celu należy zamontować dysk:
# from google.colab import drive
# drive.mount("/content/drive", force_remount=True)
# # i ustawić właściwą ścieżkę do danych w zmiennej, np.:
# data_dir = "/content/drive/My Drive/Colab Notebooks/...../"
# # konieczna będzie również modyfikacja dalszych fragmentów kodu, aby uwzględnić inną lokalizację plików

In [ ]:
import cv2
import dlib
# # w środowisku Colab biblioteka insightface wymaga zainstalowania
# !pip install insightface
# !pip install onnxruntime
import insightface 

# from google.colab.patches import cv2_imshow  # tylko w środowisku Colab
from PIL import Image
from IPython.display import display

## Obraz testowy

In [ ]:
img = cv2.imread("2_Demonstration_Demonstration_Or_Protest_2_58.jpg")
# przygotowanie obrazów: monochromatycznego i RGB (cv2.imread() zwraca obraz w formacie BGR - inna kolejność składowych)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# cv2_imshow(img)  # tylko w środowisku Colab
display(Image.fromarray(img_rgb))  # display() wymaga obrazu RGB (w przeciwieństwie do cv2.imshow(), która wymaga BGR)

## Kaskada Haara

Opracowano na podstawie: https://www.pyimagesearch.com/2021/04/05/opencv-face-detection-with-haar-cascades/

In [ ]:
# utworzenie i inicjalizacja detektora
haar_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [ ]:
# wywołanie detektora dla określonego obrazu (img_gray)
# wynikiem jest lista prostokątów w formacie [x, y, width, height]
haar_results = haar_detector.detectMultiScale(img_gray, scaleFactor=1.05, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
print(len(haar_results))

# narysowanie wyników na kopii obrazu i wyświetlenie
img_haar = img_rgb.copy()
for (x, y, w, h) in haar_results:
  cv2.rectangle(img_haar, (x, y), (x+w, y+h), (0, 255, 0), 2)
display(Image.fromarray(img_haar))

## Histogram zorientowanych gradientów (HOG) z maszyną wektorów nośnych (SVM)

Opracowano na podstawie: https://www.pyimagesearch.com/2021/04/19/face-detection-with-dlib-hog-and-cnn/

In [ ]:
# utworzenie i inicjalizacja detektora
hog_svm_detector = dlib.get_frontal_face_detector()

In [ ]:
# wywołanie detektora dla określonego obrazu (img_rgb)
# wynikiem jest lista obiektów rectangle, zawierających współrzędne lewego górnego i prawego dolnego narożnika prostokąta
hog_svm_results = hog_svm_detector(img_rgb, 1)
print(len(hog_svm_results))

# narysowanie wyników na kopii obrazu i wyświetlenie
img_hog_svm = img_rgb.copy()
for rect in hog_svm_results:
	cv2.rectangle(img_hog_svm, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
display(Image.fromarray(img_hog_svm))

## Splotowa sieć neuronowa (CNN) z biblioteki dlib

Opracowano na podstawie: https://www.pyimagesearch.com/2021/04/19/face-detection-with-dlib-hog-and-cnn/

In [ ]:
# utworzenie i inicjalizacja detektora
cnn1_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')

In [ ]:
# wywołanie detektora dla określonego obrazu (img_rgb)
# wynikiem jest lista obiektów mmod_rectangle, zawierających m.in. pole rect ze współrzędnymi lewego górnego i prawego dolnego narożnika prostokąta
cnn1_results = cnn1_detector(img_rgb, 1)
print(len(cnn1_results))

# narysowanie wyników na kopii obrazu i wyświetlenie
img_cnn1 = img_rgb.copy()
for res in cnn1_results:
	rect = res.rect
	cv2.rectangle(img_cnn1, (rect.left(), rect.top()), (rect.right(), rect.bottom()), (0, 255, 0), 2)
display(Image.fromarray(img_cnn1))

## InsightFace - inna sieć neuronowa

Opracowano na podstawie: https://github.com/deepinsight/insightface/tree/master/python-package

In [ ]:
# utworzenie i inicjalizacja detektora
model_name = 'buffalo_s'  # model: small (_s), medium (_m) lub large (_l)
insf_detector = insightface.app.FaceAnalysis(name=model_name, root='insightface',
                                             allowed_modules=['detection'], providers=['CPUExecutionProvider'])
insf_detector.prepare(ctx_id=0, det_size=(1024, 1024))

In [ ]:
# wywołanie detektora dla określonego obrazu
# wynikiem jest lista obiektów, zawierających m.in. pole bbox ze współrzędnymi lewego górnego i prawego dolnego narożnika prostokąta
insf_results = insf_detector.get(img)
print(len(insf_results))

# narysowanie wyników na kopii obrazu i wyświetlenie
img_insf = img_rgb.copy()
for res in insf_results:
  # współrzędne prostokątów sa zapisane jako liczby rzeczywiste - konwersja do liczb całkowitych
  rect = res.bbox.round().astype(int)
  cv2.rectangle(img_insf, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
display(Image.fromarray(img_insf))